In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras

from common import *
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from SDM.SDM import SDM

Using TensorFlow backend.


In [3]:
train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

In [4]:
sdm = SDM((480, 640, 3), split_into_parts=20)

images = sdm._prepare_images(image_test)
anwsers = sdm._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "SDM/SDM/weights/"

In [7]:
model_names = ["sdm_1e_2_90",
               "sdm_1e_4_30",
               "sdm_5e_6_92",
               "sdm_1e_5_35",
               "sdm_1e_6_30"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [SDM((480, 640, 3), split_into_parts=20) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=3, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 506ms/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 737ms/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 795ms/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 915ms/step - loss: 0.6931 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model SDM/SDM/weights/sdm_1e_2_90 behaved:
For cut-of-point 0.30000000000000004 had accuracy 53.52335200641007:
[-24 -18 -17 -18 -14 -13  -5  -1   0  -4  -1   0  -4   1  -8  -8  -8  -8
  -3  -5  -5  -4 -12 -10 -13 -16 -11 -14  -9  -9 -13 -19 -19 -11 -10 -12
  -6 -11 -14 -13 -10 -10 -14 -11  -9 -19 -20 -13 -14 -10 -14  -8  -9 -11
  -8 -12 -22 -24 -17 -21 -26 -29 -24 -15 -16 -15 -17 -19 -25 -14 -13 -19
 -16 -19 -15 -14 -13 -14 -16 -13  -7 -15 -13 -10 -12 -11  -7  -9 -12 -14
 -12 -15  -6 -22 -19 -24 -17 -21 -21 -13 -17 -16 -15 -16 -13 -14 -19 -20
 -26 -20 -16 -18 -21 -24 -24 -27 -15 -17 -10 -12 -21 -16 -14 -14 -11 -16
 -15 -16 -21 -16 -15 -21 -15 -17 -23 -16 -18 -11 -10 -13  -9 -17 -15 -18
 -20 -19 -17 -10 -15 -13 -13 -20 -15 -22 -21 -19 -17 -20 -11 -13 -15 -14
 -16 -13 -15 -20 -16 -13 -15 -12 -13 -17 -11  -8  -9 -11 -11 -13 -12  -8
 -11  -8 -13 -11 -13 -14 -17 -19  -9  -7 -16 -12 -18 -14 -15 -10 -21 -10
 -12 -15  -8  -6 -12 -12  -9 -16 -10 -21 -14 -20 -20 -17 -16 -18 -22 -21
 -14 -16 -15